# **Live Code 3**
Nama : Rahardiansyah Fatoni

Batch : RMT-027

Live Code 3 ini mencakup mengevaluasi SQL, Statistics, dan API sebagai berikut:

- Mampu mengambil data dari SQL database.

- Mampu menerapkan konsep central tendency.

- Mampu menerapkan konsep distribusi data.

- Mampu menerapkan konsep extreme value analysis.

- Memahami konsep statistik.

- Mampu membuat API sederhana dengan FastAPI

In [3]:
# Import modul
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
auth.authenticate_user()
print('Authenticated')

project_id = "proud-woods-408708"
client = bigquery.Client(project=project_id)

Authenticated


In [4]:
query = """
SELECT *
FROM `bigquery-public-data.iowa_liquor_sales.sales`
LIMIT 5000
"""
# Set up the query
df = client.query(query).to_dataframe()
df.head()

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,...,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,INV-22726700021,2019-10-22,5504,NEIGHBORHOOD MART,2102 LAFAYETTE ST,WATERLOO,50703,POINT(-92.313923 42.489893),7,BLACK HAWK,...,100510,RUMCHATA MINICHATAS HOLIDAY MUG,12,25,5.00,7.50,1,7.50,0.02,0.0
1,INV-23341800020,2019-11-19,5440,QUIK-PIK,502 WEST ERIE ST.,MISSOURI VALLEY,51555.0,POINT(-95.900592 41.556396),43,HARRISON,...,100794,RUMCHATA MINICHATAS SUMMER TUMBLER VAP,12,25,5.00,7.50,1,7.50,0.02,0.0
2,INV-61319400030,2023-08-16,5822,CASEY'S GENERAL STORE #2836 / MONROE,509 S MONROE ST,MONROE,50170,POINT(-93.10249604 41.51755801),None,JASPER,...,65259,JAGERMEISTER LIQUEUR MINI MEISTERS,12,20,5.63,8.45,1,8.45,0.02,0.0
3,INV-21000300022,2019-08-02,4537,CASEY'S GENERAL STORE #1834 / LEON,401 NW CHURCH ST,LEON,50144,POINT(-93.747901 40.742126),27,DECATUR,...,65259,JAGERMEISTER MINI MEISTERS,12,20,4.93,7.40,1,7.40,0.02,0.0
4,INV-23197200030,2019-11-12,4456,KUM & GO #53 / IOWA CITY,955 MORMAN TREK BLVD,IOWA CITY,52246.0,POINT(-91.574075 41.649946),52,JOHNSON,...,65259,JAGERMEISTER LIQUEUR MINI MEISTERS,12,20,4.93,7.40,1,7.40,0.02,0.0


In [5]:
query = """
SELECT sale_dollars
FROM
  bigquery-public-data.iowa_liquor_sales.sales
LIMIT 5000
"""
# Set up the query
df = client.query(query).to_dataframe()
df

,sale_dollars
0,7.50
1,7.50
2,8.45
3,7.40
4,7.40
...,...
4995,116.10
4996,116.10
4997,116.10
4998,116.10


## **A. Analisis Anomali**

1. Perhitungan central tendency (mean, median, modus) terhadap data sebelum dideteksi adanya anomali.

In [6]:
# Pencarian mean, median, dan mode pada sebuah dataframe dapat menggunakan .mean(),.median(),.mode()
print(f"Rata-rata : {df['sale_dollars'].mean()}")
print(f"Median : {df['sale_dollars'].median()}")
print(f"Modus : {df['sale_dollars'].mode()[0]}")

Rata-rata : 28.845746000000002
Median : 17.25
Modus : 77.4


2. Cek skewness data untuk mengetahui apakah data terdistribusi normal atau tidak.

In [7]:
# Untuk mencari skew pada sebuah dataframe dapat menggunakan .skew()
print(f"Skewness : {df['sale_dollars'].skew()}")

Skewness : 2.4863560211989326


Karena data memiliki skewness diatas 1, maka dapat disimpulkan data memiliki distribusi positif skew atau mencondong ke kanan.

In [8]:
# Untuk mencari kurtosis pada sebuah dataframe dapat menggunakan .kurtosis()
print(f"Kurtosis: {df['sale_dollars'].kurtosis()}")

Kurtosis: 8.465861952618315


Karena data memiliki kurtosis diatas 0, maka tipe kurtosis nya adalah Leptokurtic.

3. Lakukan pengolahan data dengan menggunakan extreme value analysis.

In [9]:
# IQR dapat dicari dengan mengurangi hasil dari quantile(0.75) dengan quantile(0.25)
q1 = df['sale_dollars'].quantile(0.25)
q3 = df['sale_dollars'].quantile(0.75)
iqr = q3-q1

# Rumus Tukey's Rule digunakan karena skew > +-0.5
up_bound = q3 + 1.5*iqr
low_bound = q1 - 1.5*iqr

print(f'Upper Boundary: {up_bound:.2f}' )
print(f'Lower Boundary: {low_bound:.2f}')

Upper Boundary: 70.23
Lower Boundary: -25.05


4. Buat variabel baru yang menyimpan data yang sudah dibuang data anomalinya.

In [16]:
outlier = df['sale_dollars'][(df['sale_dollars']<low_bound) | (df['sale_dollars']>up_bound)]
no_outlier = df['sale_dollars'][(df['sale_dollars']>=low_bound) & (df['sale_dollars']<=up_bound)]
no_outlier = pd.DataFrame(no_outlier)
no_outlier

,sale_dollars
0,7.50
1,7.50
2,8.45
3,7.40
4,7.40
...,...
4960,1.50
4961,1.50
4962,1.50
4963,32.04


**Insight**: Jumlah data outlier 707 rows.

In [15]:
print(f"Rata-rata : {no_outlier['sale_dollars'].mean()}")
print(f"Median : {no_outlier['sale_dollars'].median()}")
print(f"Modus : {no_outlier['sale_dollars'].mode()[0]}")

Rata-rata : 18.648411367342185
Median : 15.0
Modus : 38.7


**Insight**: Setelah outlier dihilangkan, didapat pemusatan data sedikit mencondong lebih ke kiri.

In [17]:
# Pembuatan csv baru dari data yang outlier nya sudah dihilangkan
no_outlier.to_csv('rahardiansyah_fatoni.csv', index = False)

## **C. Pertanyaan**

1) Berapa rata-rata, median, dan modus dari data tersebut sebelum dihilangkan outliernya? Bagaimana kecerendungan pemusatan datanya? jelaskan jawabanmu!

**Ans:**

Data memiliki rata-rata, median, dan modus sebagai berikut:

Rata-rata : 28.845746000000002

Median : 17.25

Modus : 77.4

Data memiliki kecerendungan memusat di kanan.

2) Sebelum melakukan extreme value analysis, kamu harus melakukan pengecekan skewness dari distribusi datanya. Apakah datanya skew atau normal? jelaskan jawabanmu!

Ans:

Data memiliki nilai skew 2.4863560211989326, yaitu diatas 1, yang berarti data merupakan tipe skew positif atau mencondong ke kanan.

3) Ada dua teknik untuk melakukan extreme value analysis, teknik yang mana yang kamu pakai? berikan alasanmu berdasarkan data!

Ans:

Teknik extreme value analysis menggunakan Tukey's Rule digunakan karena skewness value > 0.5 yaitu:

Upper boundary: Q3 + 1.5*IQR

Lower boundary: Q1 - 1.5*IQR

IQR = Q3 - Q1